### Functions and Objects
In Julia, a function is an object that maps a tuple of argument values to a return value. Julia functions are not pure mathematical functions, because they can alter and be affected by the global state of the program. 

We've seen println(), range(), mod(), etc...

But lets cover writing and calling functions in Julia

This is the basic syntax for defining a function

In [53]:
function f(x, y)
    x + y
end

f (generic function with 1 method)

This function accepts two arguments x and y and returns the value of the last expression evaluated, which is x + y.

There is a second, more terse syntax for defining a function in Julia. 

In [54]:
f(x, y) = x + y

f (generic function with 1 method)

A function is called using parentheses

In [55]:
f(2, 3)

5

Without parentheses, the exression refers to the function generic function object

In [56]:
g = f;
g(2, 3)

5

In [58]:
∑(x, y) = x + y
∑(2, 3)

5

### Operators are Functions
Operators are functions that have support for special syntax ( with the exception being are operators with special evaluation semanctics like /, ^, &&, and || ).

In [95]:
+(1, 2, 3)
*(1, 2, 3)

# infix form
infix = +;
infix(1, 2, 3)

6

#### The return keyword.
 the return keyword causes a function to return immediately, providing an expression whose value is returned.

The value returned by a function is the value of the last expression evaluated, which, by default, is the last expression in the body of the function definition as we've seen from previous examples of defining and calling functions.

In [82]:
function r(x, y)
    return x * y
    x + y
end
r(2, 3)


6

##### Example of a function that computes the hypotenuse length of a right triangle with sides of length x and y
Avoiding overflow while doing so.

In [83]:
function hypot(x, y)
   x = abs(x)
   y = abs(y)
   if x > y
       r = y/x
       return x*sqrt(1 + r*r)
   end
   if y == 0
       return zero(x)
   end
   r = x/y
   return y*sqrt(1 + r*r)
end

hypot(3, 4)

5.0

#### Return type
A return type can be specified in the function declaration using the :: operator

In [86]:
function gt(x, y)::Int8
   return x * y
end;

typeof(gt(1,2))

Int8

##### NOTE: Return type declarations are rarely used in Julia. Stick to "type-stable" functions so that Julia's compiler can infer the return type automatically.

For functions that do not return a value, the return keyword implicitly returns nothing, so it can be used alone. nothing can be used alone when it's the last expression. The preference is a matter of code style

In [87]:
function printx(x)
    return
end
function printx(x)
    nothing
end
function printx(x)
    return nothing
end

printx (generic function with 1 method)

### Anonymous Functions
Functions in Julia are first-class objects:
<ul>
<li>they can be assigned to variables, and called using the standard function call syntax from the variable they have been assigned to. </li>
<li>They can be used as arguments, and they can be returned as values.</li>
<li>They can also be created anonymously, without being given a name, using either of these syntaxes</li>
</ul>

In [96]:
 x -> x^2 + 2x - 1

#1 (generic function with 1 method)

In [98]:
function (x)
    x^2 + 2x - 1
end

#5 (generic function with 1 method)

##### NOTE: the result is a generic function, but with a compiler generated name based on consecutive numbering

The primary use for anonymous functions is passing them to functions which take other functions as arguments.

In [102]:
map(round, [1.2, 3.5, 1.7])
# vs using lambda
map(x -> x^2 + 2x - 1, [1, 3, -1])

3-element Vector{Int64}:
  2
 14
 -2

The return type of an anonymous function cannot be specified

#### Argument Passing Behavior
Julia function arguments follow a convention sometimes called "pass-by-sharing", which means that values are not copied when they are passed to functions. 

Function arguments themselves act as new variable bindings (new "names" that can refer to values), much like assignments argument_name = argument_value, so that the objects they refer to are identical to the passed values. (This is the same behavior found in Scheme, most Lisps, Python, Ruby and Perl, among other dynamic languages.)

In [59]:
function f(x, y)
    x[1] = 42    # mutates x
    y = 7 + y    # new binding for y, no mutation
    return y
end

f (generic function with 1 method)

##### NOTE: The behavior of a mutating function can be unexpected when a mutated argument shares memory with another argument, a situation known as aliasing

#### Argument-type declarations
Just like usual for type declarations in julia, by appending ::TypeName to the argument name.

In [60]:
fib(n::Integer) = n ≤ 2 ? one(n) : fib(n-1) + fib(n-2)

fib (generic function with 1 method)

##### NOTE:  Julia avoids automatically specializing on argument type parameters in three specific cases: Type, Function, and Vararg. Julia will always specialize when the argument is used within the method, but not if the argument is just passed through to another function. 

In [62]:
# will not specialize
f_vararg(x::Int...) = tuple(x...)
function f_type(t)  # or t::Type
    x = ones(t, 10)
    return sum(map(sin, x))
end
f_func(f, num) = ntuple(f, div(num, 2))
g_func(g::Function, num) = ntuple(g, div(num, 2))

# but this will
g_vararg(x::Vararg{Int, N}) where {N} = tuple(x...)
h_func(h::H, num) where {H} = ntuple(h, div(num, 2))
function g_type(t::Type{T}) where T
    x = ones(T, 10)
    return sum(map(sin, x))
end

g_type (generic function with 1 method)

##### One only needs to introduce a single type parameter to force specialization, even if the other types are unconstrained.

In [63]:
# this will also specialize, and is useful when the arguments are not all of the same type
h_vararg(x::Vararg{Any, N}) where {N} = tuple(x...)

h_vararg (generic function with 1 method)

**code_typed(f, types; kw...)**
Returns an array of type-inferred lowered form (IR) for the methods matching the given generic function and type signature.

In [74]:
code_typed(+, (Float64, Float64))

1-element Vector{Any}:
 CodeInfo(
1 ─ %1 = Base.add_float(x, y)::Float64
└──      return %1
) => Float64

In [76]:
# or use the macro
@code_typed f_vararg()

CodeInfo(
1 ─     return ()
) => Tuple{}

In [77]:
@code_typed g_vararg()

CodeInfo(
1 ─     return ()
) => Tuple{}

## Set-like Collections
Sets are similar to Arrays, Arrays similarly implement <b>in, union, intersect, setdiff, unique</b>

Sets are mutable containers that provide fast membership testing.

Sets have efficient implementations of set operations such as <b> in, union,</b> and <b> intersect</b>; Also some special functions for Set-like collections:
<pre>setdiff(s, itrs...) Construct the set of elements in s but not in itrs</pre>
<pre>symdiff(s, itrs...) Construct the symmetric difference of elements in the passed sets</pre>
<pre>issubset(a, b) -> Bool Determine whether every element of a is also in b, using in</pre>
<pre>⊈(a, b) -> Bool 
⊉(b, a) -> Bool

Negation of ⊆ and ⊇, i.e. checks that a is not a subset of b.</pre>
<pre>⊊(a, b) -> Bool
⊋(b, a) -> Bool

Determines if a is a subset of, but not equal to, b.</pre>
<pre>issetequal(a, b) -> Bool

Determine whether a and b have the same elements. Equivalent to a ⊆ b && b ⊆ a but more efficient when possible.</pre>
<pre>
isdisjoint(a, b) -> Bool

Determine whether the collections a and b are disjoint. Equivalent to isempty(a ∩ b) but more efficient when possible.
</pre>

In [1]:
 s = Set("aaBca")

Set{Char} with 3 elements:
  'a'
  'c'
  'B'

In [2]:
push!(s, 'b')

Set{Char} with 4 elements:
  'a'
  'c'
  'b'
  'B'

In [3]:
s = Set([NaN, 0.0, 1.0, 2.0]);

In [4]:
-0.0 in s # isequal(0.0, -0.0) is false

false

In [5]:
NaN in s # isequal(NaN, NaN) is true

true

In [4]:
a = Set([1, 3, 4, 5]);
setdiff!(a, 1:2:6);
a

Set{Int64} with 1 element:
  4

In [1]:
issubset([1, 2], [1, 2, 3])

true

In [2]:
symdiff([1,2,1], [2, 1, 2])

Int64[]

In [5]:
(1, 2) ⊈ (2, 3)

true

In [11]:
(1, 2) ⊈ (1, 2, 3)

false

In [6]:
(1, 2) ⊊ (1, 2, 3)

true

In [10]:
(1, 2) ⊊ (1, 2)

false

In [7]:
issetequal([1, 2], [2, 1])

true

In [8]:
isdisjoint([1, 2], [2, 3, 4])

false

In [9]:
isdisjoint([3, 1], [2, 4])

true

In the example below, the values are all isequal so they get overwritten in the ordinary Set. The IdSet compares by === and so preserves the 3 different values.

In [18]:
print(Set(Any[true, 1, 1.0]))
IdSet{Any}(Any[true, 1, 1.0])

Set(Any[1.0])

IdSet{Any} with 3 elements:
  true
  1
  1.0

In [17]:
[BitSet() for i in 1:6, j in 1:6]
# BitSet is basically a bitstring of flags. Use it when your sets are dense.

6×6 Matrix{BitSet}:
 BitSet([])  BitSet([])  BitSet([])  BitSet([])  BitSet([])  BitSet([])
 BitSet([])  BitSet([])  BitSet([])  BitSet([])  BitSet([])  BitSet([])
 BitSet([])  BitSet([])  BitSet([])  BitSet([])  BitSet([])  BitSet([])
 BitSet([])  BitSet([])  BitSet([])  BitSet([])  BitSet([])  BitSet([])
 BitSet([])  BitSet([])  BitSet([])  BitSet([])  BitSet([])  BitSet([])
 BitSet([])  BitSet([])  BitSet([])  BitSet([])  BitSet([])  BitSet([])

## Tuples, NTuples, and NamedTuples
Julia has a built-in data structure called a tuple that is closely related to function arguments and return values. A tuple is a fixed-length container that can hold any values, but cannot be modified
Tuples are similar to Array, but there are important differences.
<ul>
    <li>Arrays can be enclosed in square brackets while tuples are enclosed in parentheses or you can leave out the parentheses! Or simply use the tuple() type constructor</li>
    <li>Arrays are <b>mutable</b> which means you can change an element in a array, add to the array or delete an element.  Tuples are <b>immutable</b> so once a tuple is created its elements cannot be changed.</li>
    <li>Tuples are <b>heterogeneous</b>, which means they can hold elements of a different data types. Arrays are typically <b>homogeneous</b> and contain elements of a single type because storing different types in an array can lead to performance penalties.</li>
</ul>
<pre>v::Vector{Any}  = [3, 5, 8, "zwolf"] # not recommended use tuple instead</pre> 

<li>if all element for a tuple are the same type, then a <b>NTuple</b> is constructed. <li>A compact way of representing the type for a tuple of length N where all elements are of the same type.</li></li>


In [23]:
x = (1,)
print(x[1])
tuple(1)

1

(1,)

In [21]:
t1 = (3, 5, 8, "zwolf")
typeof(t1)

Tuple{Int64, Int64, Int64, String}

In [20]:
t2 = 3, 7, 5
typeof(t2)

Tuple{Int64, Int64, String}

In [17]:
isa((1, 2, 3, 4, 5, 6), NTuple{6, Int})

true

In [13]:
length(t1)

4

In [12]:
length(3)

1

In [14]:
sum(t1)

27

In [15]:
sum(3)

3

In [21]:
typeof(t1)

NTuple{4, Int64}

In [20]:
eltype(t1)

Int64

### Named Tuples
The components of tuples can optionally be named, in which case a named tuple is constructed:


In [45]:
x = (a=2, b=1+2)

(a = 2, b = 3)

In [52]:
x.a

2

## Set-like Collection functions


In [22]:
union(t1, t2)

5-element Vector{Int64}:
  3
  5
  8
 11
  7

In [23]:
intersect(t1, t2)

2-element Vector{Int64}:
 3
 5

In [44]:
t1 = [1, 2, 3, 5, 7] # Array implementations
t2 = [3, 56:100, "end"]
print(typeof(t1))
print("<= prioritize stable types; reminder that Any is unstable type =>", typeof(t2))
union(t1, t2)


Vector{Int64}<= prioritize stable types; reminder that Any is unstable type =>Vector{Any}

7-element Vector{Any}:
 1
 2
 3
 5
 7
  56:100
  "end"

In [27]:
intersect(t1,t2)

1-element Vector{Any}:
 3

## Julia Types

Julia's type system is dynamic, where nothing is known about types until run time, when the actual values manipulated by the program are available. Although though there is a camp of static type, where a program expression has a type computable before the execution of the program.

We have seen several types now starting with Int, Int64, AbstractArray, AbstractRange, Sets, and Vectors. Julia's type system is powerful and expressive.

All of these types are either Abstract types, Primitive types, or Type Aliases

<h5>NOTE: Only values, not variables, have types – variables are simply names bound to values, although for simplicity we may say "type of a variable" as shorthand for "type of the value to which a variable refers"</h5>

To get the type of a value:

<pre> typeof(x) </pre>

To determine the type of elements generated by iterating a collection of the given type.

<pre> eltype(x) </pre>

### Abstract Types

In [12]:
print(typeof(2.))
typeof(2)
# More Int8, UInt8, Int16, 
# UInt16, Int32, UInt32, 
# Int64, UInt64, Int128, 
# UInt128, Float16, Float32,
# and Float64


Float64

Int64

Abstract Types are declared using the <b>abstract type</b> keyword.

It introduces a new abstract type, whose name is given by «name»

Optionally can be followed by **<:** and an already existing type, indicating that the newly declared abstract type is a subtype of this "parent" type

<h5>Lets consider some of the abstract types that make up Julia's numerical hierarchy</h5>

In [13]:
abstract type Number end # direct child type of Any
abstract type Real          <: Number end
abstract type AbstractFloat <: Real end
abstract type Integer       <: Real end
abstract type Signed        <: Integer end
abstract type Unsigned      <: Integer end

In [16]:
#  <: operator in general means "is a subtype of"
print(Integer <: Number)
#  also be used in expressions as a subtype operator 
# which returns true when its left operand is a subtype of its right operand
Integer <: AbstractFloat

true

false

In [8]:
IntsAndFPs = Int8, UInt8, Int16, UInt16, UInt32, Int64, UInt64, Int128, Float16, Float32, Float64
eltype(IntsAndFPs)

DataType

In [9]:
typeof(IntsAndFPs)

NTuple{11, DataType}

## Strings

In [27]:
s1 = 'a'
s2 = "here is a string"

println(s1)
println(s2)

s3 = 'here'

a
here is a string


LoadError: ParseError:
[90m# Error @ [0;0m]8;;file://C:/Users/jph6366/Desktop/JULIA/In[27]#7:7\[90mIn[27]:7:7[0;0m]8;;\

s3 = '[48;2;120;70;70mhere[0;0m'
[90m#     └──┘ ── [0;0m[91mcharacter literal contains multiple characters[0;0m

In [35]:
s3 = """Here is a string that spans 
a total of three 
lines.
"""

"Here is a string that spans \na total of three \nlines.\n"

In [28]:
for c in s2
    println(c)
end

h
e
r
e
 
i
s
 
a
 
s
t
r
i
n
g


In [30]:
s2[6]

'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)

In [41]:
s4 = " here we goin\" "

" here we goin\" "

In [43]:
split(s2, "a")

2-element Vector{SubString{String}}:
 "here is "
 " string"

In [49]:
split(s2, " ", limit=2)

2-element Vector{SubString{String}}:
 "here"
 "is a string"

In [52]:
st2 = "I like to write JavaScript and I like to wash dishes"

replace(st2, "like" => "don't like")

"I don't like to write JavaScript and I don't like to wash dishes"

In [53]:
startswith(st2, "I")

true

In [54]:
endswith(st2, "dishes")

true

In [55]:
findfirst("JavaScript", st2)

17:26

In [56]:
getindex(st2, 17:26)

"JavaScript"

## Exercises

1. Write julia that finds the number of words in ex2.  Recall that the string method split() breaks the string into pieces and returns a list whose items are the pieces. This is a one-liner.
2. Use a for loop to find the average length of the words in ex2. You can use the list returned by split() to control the for loop. The average is $4.\overline{54}$.
3. Use a for loop to count the words in ex3 that start with either 'e' or 'E'.

In [103]:
# Strings for Exercises

ex2 = "This string object has a number of words separated by spaces"

ex3 = """
Aliquam erat volutpat. Sed varius luctus cursus. Sed in tempor mi.
Etiam finibus cursus arcu, non posuere dui pulvinar sit amet.
Pellentesque ultrices ante et urna ornare pretium. Duis ligula metus,
tincidunt nec euismod sit amet, porta vitae augue. Etiam a nunc nibh.
Suspendisse potenti. Aliquam sollicitudin massa ut eros egestas ullamcorper.
Nunc quis risus nibh. 
"""

"Aliquam erat volutpat. Sed varius luctus cursus. Sed in tempor mi.\nEtiam finibus cursus arcu, non posuere dui pulvinar sit amet.\nPellentesque ultrices ante et urna ornare pretium. Duis ligula metus,\ntincidunt nec euismod sit amet, porta vitae augue. Etiam a nunc nibh.\nSuspendisse potenti. Aliquam sollicitudin massa ut eros egestas ullamcorper.\nNunc quis risus nibh. \n"

In [61]:
length(split(ex2))

11

In [62]:
sum([length(i) for i in split(ex2)]) / length(split(ex2))

4.545454545454546

In [69]:
sum([startswith(uppercase(i), ("E")) for i in split(ex3)])

7